In [1]:
import paramiko
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from io import StringIO

In [ ]:
hostname = 'bridges2@'
username = 'your_username'
password = 'your_password'
remote_path = 'remote_path_to_your_csv.csv'

client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(hostname, username=username, password=password)

sftp = client.open_sftp()
with sftp.open(remote_path, 'r') as remote_file:
    csv_content = remote_file.read()

data = pd.read_csv(StringIO(csv_content.decode('utf-8')))

sftp.close()
client.close()

poly = PolynomialFeatures(degree=3)
X_poly = poly.fit_transform(data['Age'].values.reshape(-1, 1))
poly_reg_model = LinearRegression()
poly_reg_model.fit(X_poly, data['FA'].values)
trendline = poly_reg_model.predict(poly.fit_transform(np.linspace(data['Age'].min(), data['Age'].max(), 100).reshape(-1, 1)))

plt.scatter(data['Age'], data['FA'], color='skyblue', alpha=0.6, edgecolor='black', label='Data Points')

plt.plot(np.linspace(data['Age'].min(), data['Age'].max(), 100), trendline, color='blue', linestyle='-', linewidth=2, label='Trend Line')

plt.plot(np.linspace(data['Age'].min(), data['Age'].max(), 100), trendline - 0.02, color='black', linestyle='dashed', linewidth=1, label='Adjusted Trend')

plt.title('Genus of the Corpus Callosum')
plt.xlabel('Age (years)')
plt.ylabel('Fractional Anisotropy (FA)')
plt.grid(True)
plt.legend()

plt.show()
